In [1]:
import pandas as pd
import numpy as np
import os
import glob
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.model_selection import train_test_split,GridSearchCV
import pickle
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import accuracy_score, roc_curve, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import json
# from sklearn.ensemble import CatBoostClassifier

In [2]:
client = bigquery.Client()

print("Client creating using default project: {}".format(client.project))
query = """
 select * from `jm-ebg.EDA_MODEL.lable_data_with_extra_features` where event_date not like '202303%'
 """
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    # location="US",
)  # API request - starts the query
df = query_job.to_dataframe()
df.shape
# df.to_csv('train_data.csv')

Client creating using default project: jm-ebg


(88238, 28)

In [3]:
X1= df.drop(['label'], axis=1)
y1 = df.label

# Production 

In [4]:
import pandas as pd
df3 = pd.read_csv('1_2may_testing_labeled.csv',dtype={"user_pseudo_id":str})
df3.shape

(12091, 22)

In [5]:
int_cols = list(df3.select_dtypes(include=[int]).columns)
int_cols.remove('label')

In [6]:
cat_cols = list(df3.select_dtypes(include=['O']).columns)
float_cols = list(df3.select_dtypes(include=[float]).columns)

In [7]:
data={}
unwanted = {'(none)','(direct)','(not set)','(Other)'}  

In [8]:
def analysis_count(column_name,number):
    return df3.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number).reset_index(drop=True)
def analysis_column(column_name,number):
    return df3.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number)[column_name].tolist()

In [9]:
##data -> dictionary 
# data['placement'] =  analysis_column("placement",150)
# data['placement'] = [i for i in data['placement']  if i not in unwanted]
data['device_category_past'] =  analysis_column('device_category_past',4)
data['device_category_past'] = [i for i in data['device_category_past']  if i not in unwanted] 
data['mobile_brand_name_past'] =  analysis_column('mobile_brand_name_past',10)
data['mobile_brand_name_past'] = [i for i in data['mobile_brand_name_past']  if i not in unwanted] 
data['operating_system_past'] =  analysis_column('operating_system_past',5)
data['operating_system_past'] = [i for i in data['operating_system_past']  if i not in unwanted] 
data['os_version_past'] =  analysis_column('os_version_past',10)
data['os_version_past'] = [i for i in data['os_version_past']  if i not in unwanted] 
data['browser_past'] =  analysis_column('browser_past',8)
data['browser_past'] = [i for i in data['browser_past']  if i not in unwanted]
data['city_past'] =  analysis_column('city_past',160)
data['city_past'] = [i for i in data['city_past']  if i not in unwanted]
data['region_past'] =  analysis_column('region_past',50)
data['region_past'] = [i for i in data['region_past']  if i not in unwanted] 
data['ft_campaign_past'] =  analysis_column('ft_campaign_past',70)
data['ft_campaign_past'] = [i for i in data['ft_campaign_past']  if i not in unwanted]
data['ft_medium_past'] =  analysis_column('ft_medium_past',5)
data['ft_medium_past'] = [i for i in data['ft_medium_past']  if i not in unwanted] 
data['ft_source_past'] =  analysis_column('ft_source_past',10)
data['ft_source_past'] = [i for i in data['ft_source_past']  if i not in unwanted] 
data['utm_term_placement_past'] =  analysis_column('utm_term_placement_past',200)
data['utm_term_placement_past'] = [i for i in data['utm_term_placement_past']  if i not in unwanted]
# data['placement_past'] =  analysis_column('placement_past',150)
# data['placement_past'] = [i for i in data['placement_past']  if i not in unwanted]

In [10]:
# df['placement'].loc[df['placement'].apply(lambda x: (x not in data['placement']) )] = '(Others)'
df3['device_category_past'].loc[df3['device_category_past'].apply(lambda x: (x not in data['device_category_past']) )] = '(Others)'
df3['mobile_brand_name_past'].loc[df3['mobile_brand_name_past'].apply(lambda x: (x not in data['mobile_brand_name_past']) )] = '(Others)'
df3['operating_system_past'].loc[df3['operating_system_past'].apply(lambda x: (x not in data['operating_system_past']) )] = '(Others)'
df3['os_version_past'].loc[df3['os_version_past'].apply(lambda x: (x not in data['os_version_past']) )] = '(Others)'
df3['browser_past'].loc[df3['browser_past'].apply(lambda x: (x not in data['browser_past']) )] = '(Others)'
df3['city_past'].loc[df3['city_past'].apply(lambda x: (x not in data['city_past']) )] = '(Others)'
df3['region_past'].loc[df3['region_past'].apply(lambda x: (x not in data['region_past']) )] = '(Others)'
df3['ft_campaign_past'].loc[df3['ft_campaign_past'].apply(lambda x: (x not in data['ft_campaign_past']) )] = '(Others)'
df3['ft_medium_past'].loc[df3['ft_medium_past'].apply(lambda x: (x not in data['ft_medium_past']) )] = '(Others)'
df3['ft_source_past'].loc[df3['ft_source_past'].apply(lambda x: (x not in data['ft_source_past']) )] = '(Others)'
df3['utm_term_placement_past'].loc[df3['utm_term_placement_past'].apply(lambda x: (x not in data['utm_term_placement_past']) )] = '(Others)'
# df['placement_past'].loc[df['placement_past'].apply(lambda x: (x not in data['placement_past']) )] = '(Others)'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [11]:
### replace the string int and float value
for i in cat_cols:
    df3[i].fillna("Value_Not_Found", inplace = True)
for i in int_cols:
    df3[i].fillna(-99999, inplace = True) 
for i in float_cols:
    df3[i].fillna(-99999999.000, inplace = True) 

In [12]:
#categorical value handling
le = LabelEncoder()
for  i in cat_cols:
    df3[i] = le.fit_transform(df3[i])
    pickle.dump(le, open("label_encode/{}-esd-RF.pkl".format(i), 'wb'))

In [13]:
drop_cols = ['user_pseudo_id', 'visitStartTime', 'event_date','device_category_past']
df3 = df3.drop(drop_cols, axis=1)

In [14]:
df3.rename(columns={'ProspectId': 'ProspectID_LeadCreation'}, inplace=True)

In [15]:
df3.columns

Index(['ProspectID_LeadCreation', 'day_of_week_past', 'timeOnSite_past',
       'mobile_brand_name_past', 'operating_system_past', 'os_version_past',
       'browser_past', 'region_past', 'city_past', 'ft_campaign_past',
       'ft_source_past', 'ft_medium_past', 'utm_term_placement_past',
       'mx_Last_Activity_Notes', 'ProspectActivityName_Max',
       'mx_Last_Activity_done_by_Agent', 'mx_Sub_Dispostion', 'label'],
      dtype='object')

In [16]:
X = df3.drop(['label'], axis=1)
y = df3.label

In [19]:
from sklearn.preprocessing import StandardScaler

ss_train = MinMaxScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = MinMaxScaler()
X_test = ss_test.fit_transform(X_test)

In [23]:
X.head()

,ProspectID_LeadCreation,day_of_week_past,timeOnSite_past,mobile_brand_name_past,operating_system_past,os_version_past,browser_past,region_past,city_past,ft_campaign_past,ft_source_past,ft_medium_past,utm_term_placement_past,mx_Last_Activity_Notes,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
0,0,3,0.000000,3,0,3,2,4,68,0,0,0,0,421,9,3,46
1,1,2,1.474086,0,0,0,2,30,47,15,6,1,0,159,10,5,40
2,1,2,0.188153,0,0,0,2,30,47,15,6,1,0,159,10,5,40
3,1,2,1.897437,0,0,0,2,30,47,15,6,1,0,159,10,5,40
4,1,2,21.324201,0,0,0,2,30,47,15,6,1,0,159,10,5,40


In [18]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=.25, stratify=y)

In [22]:
X.loc[:,['timeOnSite_past', 'day_of_week_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'utm_term_placement_past',
       'ProspectID_LeadCreation', 'mx_Last_Activity_Notes',
       'ProspectActivityName_Max', 'mx_Last_Activity_done_by_Agent',
       'mx_Sub_Dispostion']]

,timeOnSite_past,day_of_week_past,mobile_brand_name_past,operating_system_past,os_version_past,browser_past,region_past,city_past,ft_campaign_past,ft_medium_past,ft_source_past,utm_term_placement_past,ProspectID_LeadCreation,mx_Last_Activity_Notes,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
0,0.000000,3,3,0,3,2,4,68,0,0,0,0,0,421,9,3,46
1,1.474086,2,0,0,0,2,30,47,15,1,6,0,1,159,10,5,40
2,0.188153,2,0,0,0,2,30,47,15,1,6,0,1,159,10,5,40
3,1.897437,2,0,0,0,2,30,47,15,1,6,0,1,159,10,5,40
4,21.324201,2,0,0,0,2,30,47,15,1,6,0,1,159,10,5,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12086,0.070109,3,5,0,8,2,10,3,5,1,6,0,1855,109,12,5,36
12087,0.793212,3,5,0,8,2,10,3,5,1,6,0,1855,109,12,5,36
12088,0.793212,3,5,0,8,2,10,3,5,1,6,40,1855,109,12,5,36
12089,0.793212,3,5,0,8,2,10,3,5,1,6,0,1855,109,12,5,36


In [24]:
filename = 'model/RF_Dhawal.pkl'
model = pickle.load(open(filename, 'rb'))

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score

y_pred = model.predict(X_train)
#y_pred = [1 if val >= 0.50 else 0 for val in y_pred_prob[:,1]]
f1 = f1_score(y_train, y_pred)
print("accuracy : ", accuracy_score(y_train, y_pred))
print(confusion_matrix(y_train, y_pred))
print("roc auc : ", roc_auc_score(y_train, y_pred))
print("recall : ", recall_score(y_train, y_pred))
print("precision : ", precision_score(y_train, y_pred))
print("f1 score : ",f1 )

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


accuracy :  0.9913983237759153
[[8990    0]
 [  78    0]]
roc auc :  0.5
recall :  0.0
precision :  0.0
f1 score :  0.0


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
y1=y1.astype('int64')
y=y.astype('int64')